# CS4248 Assignment 3

### Perceptron text classifier in Python with feature selection

##### https://github.com/jia1/pyceptron

In [1]:
import math
import os
import re
import sys
from string import punctuation
from porter import PorterStemmer

p = PorterStemmer()

In [2]:
src_dir = os.path.abspath('tc')

stopword_list_file = 'stopword-list'
train_class_list_file = 'train-class-list'
model_file = 'model'

In [3]:
'''
class_to_text
<class 'dict'>
KEY: <class 'str'>
VAL: <class 'set'>
E.g: {'c1': {'0000', '0001', '0002', ... }, 'c2': {'1000', '1001', '1002', ... } }

text_to_count
<class 'dict'>
KEY: <class 'str'>
VAL: <class 'dict'>
     KEY: <class 'str'>
     VAL: <class 'int'>
E.g: {'0000': {'word0': 1, 'word1': 3, 'word2': 5, ... }, '0001': { ... }}

class_to_feat_chi_tup
<class 'dict'>
KEY: <class 'str'>
VAL: <class 'list'>
     VAL: <class 'tuple'>
          <class 'str'>
          <class 'float'>
E.g: {'c1': [ ('word1', 99.00), ('word2', 90.00), ... ], 'c2': [ ... ] }

nxx_to_word_to_class_to_chi

class_to_word_to_chi

Both feat and word are either unigram or bigram
'''

k = 3
max_compromise = 1
num_both, num_train = 0, 0
train_ratio = 1
# train_ratio = 0.8
# test_ratio = 1 - train_ratio

num_class = 0
class_list = []
class_to_text = {}

In [4]:
feat_prune_ratio = 0.5

text_to_count = {}
nxx_list = ['n10', 'n11']
nxx_map = {
    'n00': 'n10',
    'n01': 'n11'
}
nxx_to_word_to_class_to_chi = { n: {} for n in nxx_list }
class_to_word_to_chi = {}
class_to_feat_chi_tup = {}

In [ ]:
feat_prune_ratio = 0.5

class_to_vocab_to_tfidf = {}

In [ ]:
feat_prune_ratio = 0.5

text_to_word_list = {}
# class_to_word_to_num_text = {}
# text_to_word_to_tfidf = {}
class_to_feat_tfidf_tup = {}

In [ ]:
class_to_feat_set = {}
class_to_feat_list_sort_by_lex = {}
class_to_feat_to_index = {}
class_to_feat_mat = {}

In [5]:
# A. Split each line by space into tokens
# B. Strip all default white space characters from each token
# C. Remove punctuation from each token
# D. Return a list of tokens which are not stop words

def strip_and_filter_line(ln):
    if all(x in ln for x in [':', '@']):
        return []
    tokens = map(lambda t: t.strip().strip(punctuation).lower(), ln.split(' '))
    return list(filter(lambda t: t and len(t) > 2 and t.isalpha() and t not in stop_list, tokens))

In [ ]:
def get_word_to_count(word_list):
    word_to_count = {}
    num_words = len(word_list)
    prev_unigram = word_list[0]
    for i in range(1, num_words)
        curr_unigram = word_list[i]
        ngrams = [curr_unigram, '{} {}'.format(prev_unigram, curr_unigram)]
        for ngram in ngrams:
            if ngram not in word_to_count:
                word_to_count[ngram] = 1
            else:
                word_to_count[ngram] += 1
        prev_unigram = curr_unigram
    return word_to_count

In [ ]:
def get_weaker_word_to_count(word_to_count):
    fin_word_to_count = {}
    for compromise in range(1, max_compromise - 1):
        if fin_word_to_count:
            break
        fin_word_to_count = { word: count for word, count in word_to_count.items() \
                             if count >= k - compromise }
        for len_gram in range(2, 0, -1):
            fin_word_to_count = { word: count for word, count in fin_word_to_count.items() \
                                 if len(word.split(' ')) >= len_gram }
            if fin_word_to_count:
                break
    return fin_word_to_count

In [6]:
def is_in(a, b):
    return 1 if a in b else 0

def count_nxx(nxx, w, c):
    nxx_value = 0
    if nxx == 'n10':
        for class_name in filter(lambda x: x != c, class_list):
            for text in class_to_text[class_name]:
                nxx_value += is_in(w, text_to_count[text])
    elif nxx == 'n11':
        for text in class_to_text[c]:
            nxx_value += is_in(w, text_to_count[text])
    return nxx_value

In [7]:
# n00 is the number of training texts that do not contain w and are not in class c.
# n01 is the number of training texts that do not contain w and are in class c.
# n10 is the number of training texts that contain w and are not in class c.
# n11 is the number of training texts that contain w and are in class c.

# n00 = num_train - n10
# n01 = num_train - n11
# n10 = num_train - n00
# n11 = num_train - n01

def calc_chi_square(w, c):
    nxx_to_count = {}
    for n in nxx_list:
        if w not in nxx_to_word_to_class_to_chi[n]:
            nxx_to_word_to_class_to_chi[n][w] = {}
        if c not in nxx_to_word_to_class_to_chi[n][w]:
            nxx_to_word_to_class_to_chi[n][w][c] = count_nxx(n, w, c)
        nxx_to_count[n] = nxx_to_word_to_class_to_chi[n][w][c]
    for n, nn in nxx_map.items():
        nxx_to_count[n] = num_train - nxx_to_word_to_class_to_chi[nn][w][c]
    n00, n01, n10, n11 = nxx_to_count['n00'], nxx_to_count['n01'], nxx_to_count['n10'], nxx_to_count['n11']
    return ((n11+n10+n01+n00)*(n11*n00-n10*n01)**2)/((n11+n01)*(n11+n10)*(n10+n00)*(n01+n00))

In [8]:
def put_chi(c, w, chi_value):
    global class_to_word_to_chi
    if w not in class_to_word_to_chi[c]:
        class_to_word_to_chi[c][w] = chi_value
    else:
        class_to_word_to_chi[c][w] = max(class_to_word_to_chi[c][w], chi_value)

In [9]:
def gen_feat_by_chi():
    global class_to_feat_chi_tup
    max_feat_vec_len = sys.maxsize
    class_to_feat_sorted = { c: [] for c in class_list }
    for c in class_to_word_to_chi:
        class_to_feat_sorted[c] = sorted(class_to_word_to_chi[c].items(), key = lambda x: x[1], reverse = True)
        max_feat_vec_len = min(max_feat_vec_len, len(class_to_feat_sorted[c]))
    max_feat_vec_len *= feat_prune_ratio 
    class_to_feat_chi_tup = { c: class_to_feat_sorted[c][:int(max_feat_vec_len)] for c in class_to_feat_sorted }

In [ ]:
def gen_feat_by_tfidf():
    global class_to_vocab_to_tfidf
    
    for c in class_list:
        for text in class_to_text[c]:
            word_list = text_to_word_list[text]
            prev_unigram = word_list[0]
            class_to_vocab_to_tfidf[c][prev_unigram] = 0
            for i in range(1, len(word_list)):
                curr_unigram = word_list[i]
                bigram = '{} {}'.format(prev_unigram, curr_unigram)
                class_to_vocab_to_tfidf[c][curr_unigram] = 0
                class_to_vocab_to_tfidf[c][bigram] = 0
                prev_unigram = curr_unigram
    for c in class_list:
        for text in class_to_text[c]:
            word_list = text_to_word_list[text]
            prev_unigram = word_list[0]
            class_to_vocab_to_tfidf[c][prev_unigram] = 0
            for i in range(1, len(word_list)):
                curr_unigram = word_list[i]
                bigram = '{} {}'.format(prev_unigram, curr_unigram)
                class_to_vocab_to_tfidf[c][curr_unigram] += 1
                class_to_vocab_to_tfidf[c][bigram] += 1
                prev_unigram = curr_unigram

    for c in class_list:
        num_texts = len(class_to_text[c])
        for v in class_to_vocab_to_tfidf[c]:
            class_to_vocab_to_tfidf[c][v] = math.log(num_texts / (1 + class_to_vocab_to_tfidf[c][v]))
                
    max_feat_vec_len = sys.maxsize
    class_to_feat_sorted = { c: [] for c in class_list }
    for c in class_to_word_to_chi:
        class_to_feat_sorted[c] = sorted(class_to_vocab_to_tfidf[c].items(), key = lambda x: x[1], reverse = True)
        max_feat_vec_len = min(max_feat_vec_len, len(class_to_feat_sorted[c]))
    max_feat_vec_len *= feat_prune_ratio 
    class_to_vocab_to_tfidf = { c: class_to_feat_sorted[c][:int(max_feat_vec_len)] for c in class_to_feat_sorted }

In [ ]:
def contains_sub_list(main_list, sub_list):
    n = len(sub_list)
    return any((sub_list == main_list[i:i+n]) for i in range(len(main_list) - n+1))

def get_num_texts_contains_word(word, class_name):
    global class_to_word_to_num_text
    if class_to_word_to_num_text[class_name][word]:
        return class_to_word_to_num_text[class_name][word]
    else:
        num_texts = 0
        ngram = word.split(' ')
        len_ngram = len(ngram)
        if len_ngram == 1:
            for text in class_to_text[class_name]:
                if word in text_to_word_list[text]:
                    num_texts += 1
            class_to_word_to_num_text[class_name][word] = num_texts
        elif len_ngram == 2:
            for text in class_to_text[class_name]:
                if contains_sub_list(text_to_word_list[text], ngram):
                    num_texts += 1
            class_to_word_to_num_text[class_name][word] = num_texts
        return num_texts

In [ ]:
# http://nlpforhackers.io/tf-idf/
# importance ~ appearances(term, document) / count(documents containing term in collection)
# tf = count(word, document) / len(document)
# idf = log(len(collection) / count(document_containing_term, collection))
# tf_idf = tf * idf

# To be implemented
def get_tfidf(word, word_list, class_name):
    word_sub_list = word.split(' ')
    tf = sum(1 for i in range(len_word_list) if word_list[i:i + len(word_sub_list)] == word_sub_list) / len_word_list
    idf = math.log2(len(class_to_text[class_name]) / get_num_texts_contains_word(word, class_name))
    return tf * idf

In [ ]:
def put_tfidf(text, word):
    global text_to_word_to_tfidf
    if text not in text_to_word_to_tfidf:
        text_to_word_to_tfidf[text] = {}
    text_to_word_to_tfidf[text][word] = get_tfidf(word, text_to_word_list[text])

In [ ]:
'''
def gen_feat_by_tfidf():
    global class_to_feat_tfidf_tup
    max_feat_vec_len = sys.maxsize
    class_to_feat_sorted = { c: [] for c in class_list }
    for c in class_to_word_to_tfidf:
        # TODO: Define class_to_word_to_tfidf
        class_to_feat_sorted[c] = sorted(class_to_word_to_tfidf[c].items(), key = lambda x: x[1], reverse = True)
        max_feat_vec_len = min(max_feat_vec_len, len(class_to_feat_sorted[c]))
    max_feat_vec_len *= feat_prune_ratio 
    class_to_feat_tfidf_tup = { c: class_to_feat_sorted[c][:int(max_feat_vec_len)] for c in class_to_feat_sorted }
'''

In [10]:
def feat_select():
    for c in class_list:
        for text in class_to_text[c]:
            for w in text_to_count[text]:
                put_chi(c, w, calc_chi_square(w, c))
    gen_feat_by_chi()

In [11]:
with open(stopword_list_file, 'r') as s:
    stop_list = list(map(lambda ln: ln.strip(), s.readlines()))

In [ ]:
with open(train_class_list_file, 'r') as t:
    lines = map(lambda ln: ln.strip().split(' '), t.readlines())
    for ln in lines:
        file, curr_class = ln
        text = file.split('/')[-1]
        num_both += 1
        num_train += 1
        flat_text = []
        with open(file, 'r') as f:
            for line in map(lambda ln: strip_and_filter_line(ln), f.readlines()):
                flat_text.extend(list(map(lambda word: p.stem(word, 0, len(word) - 1), line)))
            word_to_count = get_word_to_count(flat_text)
            fin_word_to_count = { word: count for word, count in word_to_count.items() if count >= k }
            if not fin_word_to_count:
                fin_word_to_count = get_weaker_word_to_count(word_to_count)
            sum_count = sum(fin_word_to_count.values())

            if curr_class not in class_list:
                class_list.append(curr_class)
                num_class += 1
                class_to_text[c] = set()
                class_to_word_to_chi[curr_class] = {}
                class_to_feat_chi_tup[curr_class] = set()
                class_to_vocab_to_tfidf[curr_class] = {}
                # class_to_word_to_num_text[curr_class] = {}
                class_to_feat_tfidf_tup[curr_class] = set()
                class_to_feat_set[curr_class] = set()
                class_to_feat_list_sort_by_lex[curr_class = []
                class_to_feat_to_index[curr_class = {}

            class_to_text[curr_class].add(text)
            text_to_word_list[text] = flat_text
            text_to_count[text] = { word: count / sum_count for word, count in fin_word_to_count.items() }

In [12]:
for curr_dir, sub_dir, files in os.walk(src_dir):
    if not files:
        class_list = sub_dir
        num_class = len(class_list)
        class_to_text = { c: set() for c in class_list }
        class_to_word_to_chi = { c: {} for c in class_list }
        class_to_feat_chi_tup = { c: set() for c in class_list }
        class_to_vocab_to_tfidf = { c: {} for c in class_list }
        # class_to_word_to_num_text = { c: {} for c in class_list }
        class_to_feat_tfidf_tup = { c: set() for c in class_list }
        class_to_feat_set = { c: set() for c in class_list }
        class_to_feat_list_sort_by_lex = { c: [] for c in class_list }
        class_to_feat_to_index = { c: {} for c in class_list }
        continue

    curr_class = re.split('[(\\\\)(\\)(\/)]', curr_dir)[-1] # curr_dir.split('\\')[-1]
    curr_num_files = len(files)
    num_both += curr_num_files
    curr_num_train = int(curr_num_files * train_ratio)
    num_train += curr_num_train
    for i in range(curr_num_train):
        file = files[i]
        flat_text = []
        with open(os.path.join(curr_dir, file), 'r') as f:
            for line in map(lambda ln: strip_and_filter_line(ln), f.readlines()):
                flat_text.extend(list(map(lambda word: p.stem(word, 0, len(word) - 1), line)))
            word_to_count = get_word_to_count(flat_text)
            fin_word_to_count = { word: count for word, count in word_to_count.items() if count >= k }
            if not fin_word_to_count:
                fin_word_to_count = get_weaker_word_to_count(word_to_count)
            sum_count = sum(fin_word_to_count.values())
            class_to_text[curr_class].add(file)
            text_to_count[file] = { word: count / sum_count for word, count in fin_word_to_count.items() }

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(text_to_count)

In [13]:
from time import time
start = time()

class_to_word_to_chi = { c: {} for c in class_list }
class_to_feat_chi_tup = { c: set() for c in class_list }
class_to_word_to_num_text = { c: {} for c in class_list }
class_to_feat_tfidf_tup = { c: set() for c in class_list }

feat_select()

stop = time()
print(stop - start)

48.88188815116882


In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(class_to_feat_chi_tup)

In [14]:
class_to_feat_set = { c: set() for c in class_list }

for c in class_to_feat_chi_tup:
    for p in class_to_feat_chi_tup[c]:
        w = p[0]
        class_to_feat_set[c].add(w)
    curr_num_feat = len(class_to_feat_set[c])
    num_feat_per_neg_class = curr_num_feat // (num_class - 1)
    for nc in class_to_feat_chi_tup:
        if nc != c:
            num_added = 0
            for t in class_to_feat_chi_tup:
                class_to_feat_set[c].add(t[0])
                if num_added >= num_feat_per_neg_class:
                    break

class_to_feat_list_sort_by_lex = { c: sorted(list(class_to_feat_set[c])) for c in class_list }
class_to_feat_to_index = { c: {} for c in class_list }

for c in class_to_feat_list_sort_by_lex:
    for i in range(len(class_to_feat_list_sort_by_lex[c])):
        class_to_feat_to_index[c][class_to_feat_list_sort_by_lex[c][i]] = i

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(class_to_feat_to_index)

In [15]:
# https://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/

from random import seed, randrange
seed(4248)

# Split data_mat into num_folds number of folds
def get_folds(data_mat, num_folds):
    folds = []
    data_clone = list(data_mat)
    fold_size = int(len(data_mat) / num_folds)
    for i in range(num_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(data_clone))
            fold.append(data_clone.pop(index))
        folds.append(fold)
    return folds

# Calculate accuracy percentage
def get_accuracy(predicted, actual):
    num_correct = 0
    for i in range(len(actual)):
        if predicted[i] == actual[i]:
            num_correct += 1
    return num_correct / len(actual) * 100

# Evaluate an algorithm using a cross validation split
def get_cross_validation_scores(data_mat, algorithm, num_folds, *args):
    folds = get_folds(data_mat, num_folds)
    scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = [row for fold in train_set for row in fold]
        test_set = []
        for row in fold:
            row_clone = list(row)
            test_set.append(row_clone)
            row_clone[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = get_accuracy(predicted, actual)
        scores.append(accuracy)
    return scores

# Make a prediction with weights
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row) - 1):
        activation += weights[i + 1] * row[i]
    return 1 if activation >= 0 else 0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, alpha, max_iterations = 1000):
    weights = [0 for i in range(len(train[0]))]
    for _ in range(max_iterations):
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            weights[0] = weights[0] + alpha * error
            for i in range(len(row) - 1):
                weights[i + 1] += alpha * error * row[i]
    return weights

# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, alpha, max_iterations):
    predictions = list()
    weights = train_weights(train, alpha, max_iterations)
    print(weights)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return predictions

In [16]:
# load and prepare data
class_to_feat_mat = { c: [] for c in class_list }
for c in class_list:
    for d in class_list:
        texts = class_to_text[d]
        num_texts = len(texts)
        texts = iter(texts)
        if c != d:
            num_texts_to_train = int(num_texts * train_ratio / (num_class - 1))
        else:
            num_texts_to_train = num_texts
        for i in range(num_texts_to_train):
            text = next(texts)
            feat_vec = [0 for i in range(len(class_to_feat_to_index[d]) + 1)]
            for word in text_to_count[text]:
                if word in class_to_feat_to_index[d]:
                    index = class_to_feat_to_index[d][word]
                    feat_vec[index] = text_to_count[text][word]
            feat_vec[-1] = 1 if c == d else 0
            class_to_feat_mat[c].append(feat_vec)

In [ ]:
data = class_to_feat_mat
num_folds_list = [5, 10] # [5, 10]
alpha_list = [0.05, 0.1] # [0.02, 0.03, 0.05, 0.07, 0.1]
max_iterations_list = [500, 1000, 2000] # [500, 1000, 2000]

for num_folds in num_folds_list:
    for alpha in alpha_list:
        for max_iterations in max_iterations_list:
            print('{}-fold cross validation'.format(num_folds))
            print('Learning rate: {}, maximum number of iterations: {}'.format(alpha, max_iterations))
            for c in class_list:
                scores = get_cross_validation_scores(data[c], perceptron, num_folds, alpha, max_iterations)
                print('Class: {}'.format(c))
                print('Cross validation scores: {}'.format(scores))
                print('Mean accuracy: {:.2f}%'.format(sum(scores) / num_folds))
                print()

5-fold cross validation
Learning rate: 0.05, maximum number of iterations: 500
[-1.3877787807814457e-17, -0.025423728813559327, -0.13366637241750434, -0.03089566636984272, 0.11206248050153962, 0.3761692683197338, 0.163074036382139, -0.19380962915832484, 0.10526261629988772, -0.26370123926207767, -0.1518824540730778, -0.2630409868649789, 0.39417536220617005, 0.017316725268149424, -0.13215453735934626, -0.4029067184302211, 0.02911193035196988, 0.03831826122700628, -0.049283284140414316, 0.06078451181735151, -0.007546749777381997, -0.05054945054945054, 0.075, -0.007993884080254978, -0.05390624999999998, 0.24708035846643217, -0.16404589064587793, -0.10355029585798824, -0.08925448846802743, -0.08692922123532633, 0.39517865392455515, -0.10606060606060605, -0.03875111746730209, -0.1552898589703724, -0.041666666666666664, 0.056298076923076916, -0.011315252881128416, -0.06925492529678579, -0.1711220606523971, -0.007359859714605882, -0.5274107552889421, -0.04879165659218079, -0.02674649488487532

[-1.3877787807814457e-17, 0.007520275251904615, -0.1419418651889836, 0.00489072588089923, 0.0658506473283797, 0.40749797664262205, -0.020669612748566973, -0.09364107373139921, -0.1319585034186811, -0.19415716096324456, -0.04249772907286554, -0.20983913222595096, 0.1492194318691979, 0.03243291241336448, -0.022620299391423786, -0.27509875142854195, -0.012646326666870716, 0.1607011954920685, -0.5937239542740218, 0.0022132869991253606, 0.013713125528881601, 0.0708715941592654, 0.1465454532116553, -0.05559135682449042, -0.018303571428571433, 0.12983549283345638, -0.20761537963906374, -0.13402366863905338, -0.057598750333001984, -0.004423047422540402, 0.2656450163322548, -0.11183951830916772, -0.05033484095389571, 0.056572489876386974, -0.04976693781361284, 0.12475024975024948, -0.013713362686551376, -0.11297430913831391, -0.10846594933909448, -0.08538001080825486, -0.6555527539557006, -0.13454365030547794, -0.06164794042830388, 0.11249999999999999, -0.07826086956521743, 0.04602587336638855,

[-1.3877787807814457e-17, 0.0032084258406887903, -0.10384538830131085, -0.0249092221220164, 0.14092436276373577, -0.014277763037254778, 0.12151649588901003, -0.1517334116921586, 0.07286714116099949, -0.4245004224757096, -0.045034201093334864, -0.33314886436944247, 0.227126129619876, -0.03389307162395921, -0.0482802883897673, -0.25150833884868684, 0.3181412293138526, -0.31059072380365127, -0.16749364652805188, 0.016465443940540517, 0.0062424346176261445, 0.030769230769230785, 0.19154545321165528, -0.06189993935472904, 0.0, -0.003868864902773063, -0.33496060904896524, -0.07248520710059193, -0.014310543863472872, -0.2926956382414611, 0.3990937524487544, -0.15993610223642232, -0.03615787818113349, -0.33555915546815546, -0.010665198232640009, 0.14802697302697274, -0.14500337626791363, -0.04914154860798245, -0.20501427638067077, -0.02581399564555269, -0.45530222356096167, -0.027830124702122733, 0.19949371099657218, 0.11249999999999999, -0.20625000000000016, -0.08875839998319729, -0.055188679

[-1.3877787807814457e-17, 0.002913513085712138, -0.10086729298678863, 0.008381021035111014, 0.04388445748633061, 0.5090730602265524, 0.0790600654094273, -0.25686447696071135, 0.07793054839501128, -0.23324074074074078, -0.24318014254116452, -0.3476312504630217, 0.3792621964035512, -0.054644100657952076, -0.05987338511198248, -0.2677355928236812, 0.17612805204499266, 0.04202702702702745, -0.40957964989928897, 0.08139937486210556, 0.005564995333960385, 0.026373626373626384, 0.14796054755127794, 0.005725067004286904, -0.10887706043956055, 0.20921461546367406, 0.3219660952006743, -0.10192307692307696, 0.034474522292993665, -0.14960573476702502, 0.10392394684787218, 0.009547800442369136, -0.11405575724364672, -0.18661055344906619, -0.037789526767162615, 0.048038836163836154, -0.08117636718333338, 0.01356750865288919, -0.0888298602350624, -0.04424899817601126, -1.0028449619965851, 0.003315348534574069, -0.017516365311416437, 0.11249999999999999, -0.5173295454545446, 0.06978260869565218, -0.05

[0.0, 0.0064389284836568975, -0.05662924418171277, 0.00482919636274268, 0.037778495318049735, 0.5050788591078474, 0.1569095756498556, -0.1994662596785637, 0.1307938905480057, -0.30055555555555635, -0.21981175129474229, -0.11866520738999416, 0.35699265418146864, -0.054077760497667174, 0.019406392694063926, -0.3415875952414562, 0.12948035421096254, -0.22500000000000048, -0.06110931726043202, 0.13567918633604503, 0.01755153233472215, 0.038461538461538464, -0.02269982980721265, -0.004688084160429146, -0.008816964285714287, 0.10759892507838048, -0.1818702487071736, -0.07500000000000001, -0.015000000000000041, -0.026666666666666672, 0.34284610497024964, -0.11373187515360103, -0.06153846153846154, -0.20205277623827272, 0.009631103720560578, 0.07886925574425568, 0.051502202428909565, 0.08928571428571427, -0.2850344704543772, 0.03, -0.26720871695816345, -0.015532078433895027, 0.06875318070898599, 0.037500000000000006, -0.3707880434782603, 0.052335630965005306, -0.06792452830188679, 0.1468860003

...

[0.04999999999999999, -0.03838983050847459, 0.0526096295061157, 0.3270369545859368, -0.100454260651629, -0.1334689722534153, -0.26317337084801057, 0.19505866967385468, -0.1844433198380567, -0.059522167548895256, 0.22216819221967907, -0.33951978254303844, -0.020347284060551973, -0.05860540275044558, -0.14581032947462166, 0.5891235328039444, -0.11410417719365, 0.026476552598225605, -0.02190170940170941, -0.06349675327923723, 0.2144160356432403, -0.05439560439560439, -0.0009331259720062207, -0.004044943820224719, 0.1290144230769229, -0.050067005810108596, -0.3153032995873152, -0.026075268817204282, -0.01923003802281368, -0.24682925167286765, -0.06170721337674362, 0.2144927536231884, 0.003226248373176573, -0.06809989730097067, 0.08333333333333333, 0.03482126002831525, 0.06823608162097106, -0.046424597115051335, 0.16902375002713554, -0.08703603153573809, 0.05017701261463862, 0.2274012993651937, -0.06923267051566724, 0.0075, -0.11888945556547366, 0.028793103448275854, 0.0, -0.003571428571428

[-0.05, 0.0, -0.10997866781481835, -0.02619842683288812, 0.17232089183734814, -0.02312292087242767, 0.3161526411059227, 0.03413165266106489, 0.01819462583257132, -0.11133413170361564, -0.047826086956521754, 0.626757676432095, -0.13443625667289372, 0.10827548330298138, -0.1581389136242133, 0.17582995951417055, 0.30198167825961675, 0.06083333333333349, 0.03383918837363746, 0.2570834355494849, 0.10773314327770614, 0.029355740594169004, 0.10638213530655531, 0.003433758568687031, 0.004687500000000001, 0.08226704180699916, -0.052688912806487895, -0.06904869251577994, -0.3599932953402611, -0.08978494623655875, 0.06963490650044477, -0.021735252082203907, 0.1473662579628736, -0.07565999898645023, -0.06766004415011037, -0.0019230769230769232, 0.1257539329177811, -0.03600911424231328, -0.16597962563412025, 0.13256648341328126, -0.006597349536430909, 0.10306828689368136, -0.14830806706448432, 0.00947410147991543, 0.0460716403162055, 0.018585298196948683, 0.0029756537421100093, -0.01580645161290322

[0.0, 0.0, -0.09882789875258247, -0.011875963819521872, 0.1073294852324678, -0.026584634357890165, 0.2413310846603345, 0.12330532212885215, 0.013408822537111675, -0.40709601666264145, 0.01493935926773455, 0.5026839826839825, -0.11789044781498272, 0.06714417813619679, -0.11049387966149166, -0.05352785620176877, 0.04890773592101774, 0.13977272727272722, 0.03996016150482317, 0.04859010399910645, 0.07079177046300959, -0.0367311951614278, 0.10410330121087207, 0.01629228329809725, 0.032812499999999994, 0.04953171764629865, -0.07236627622048848, -0.06012173128944987, -0.029156469996647793, -0.09311827956989221, 0.04644041806276719, -0.030647341880513365, 0.06533364700297253, 0.028117562559284194, -0.041666666666666664, -0.08442307692307698, 0.09424723447072375, -0.003576090511700304, -0.17556274973045327, 0.21254500025963496, 0.02289449772359382, -0.05104015496836612, -0.0780949391501636, 0.0054959784906700995, 0.11822282608695664, 0.0004016271438719368, 0.0074391343552750126, 0.0, 0.27201135

[0.0, 0.0, -0.04242111288908031, 0.0046230335411101406, 0.10792771773889168, -0.0010423247182560253, 0.2921187035506599, 0.10543417366946842, 0.012251602564102296, -0.4215700876081782, 0.008953089244851262, 0.4266060606060605, -0.18380429844823226, 0.09656283378709737, -0.029826357969725763, -0.029047619047618604, 0.2953299380624605, 0.06992424242424242, 0.07175453526194173, 0.1913790366869339, 0.09839581946093222, -0.009337586850569613, 0.09923361522198858, -0.04915516372151131, 0.0023437500000000003, 0.0746899820964304, -0.03932816745134401, -0.011744815148782728, -0.31970331880657016, -0.10422939068100348, 0.06295636687444266, -0.04802400078852699, 0.18886910062332984, 0.04433109629404385, -0.025993377483443707, -0.03754807692307694, 0.15739091718610995, 0.04721727515583246, 0.1321385674890176, 0.01887220338604298, -0.01160625425593223, 0.14485801774391674, -0.08615926694209342, 0.008800211416490494, 0.07876828063241112, 0.006173082777984503, 0.006627592425608649, -0.015806451612903

[0.0, -0.012711864406779662, -0.11011779673385842, -0.0016685761834235278, 0.16690247654754534, -0.028818225606710563, 0.2801436128330865, 0.0646078431372556, 0.02307692307692278, -0.1801773549447968, 0.01132723112128143, 0.607186147186147, -0.07907186360964258, 0.13180856018222584, 0.15026925903119423, -0.09095238095238031, 0.3020282216356767, 0.1483333333333333, 0.37998341604402075, 0.1732818638886411, 0.10384131014218174, -0.0115577214611986, 0.010019318203321116, -0.02088950030482203, 0.0, 0.0936538339645687, -0.057883562076185016, -0.0528178539224526, -0.40460526315789536, -0.07546594982078828, 0.08876063455701637, -0.02420999457887733, 0.08826350221895964, 0.05306694765337298, -0.041666666666666664, -0.06000000000000004, 0.08324007296699103, 0.029079450215373392, -0.14052792651333956, 0.1530118416072189, 0.0055500384506675165, 0.0618520505066107, -0.11941895022452279, 0.00606443606949169, 0.17263315217391317, -0.025021396434491458, 0.011091073038773649, -0.024838709677419354, 0.1

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(class_to_feat_mat)

In [ ]:
import ast
import re

from string import punctuation
from porter import PorterStemmer

p = PorterStemmer()

stopword_list_file = 'stopword-list'
# test_list_file, test_class_list_file = 'test-list', 'test-class-list'
test_list_file, test_class_list_file = 'train-class-list-copy', 'train-class-list-copy'
model_file = 'model'

with open(model_file, 'r') as m:
    lines = list(map(lambda w: ast.literal_eval(w), m.readlines()))
    class_list, class_to_feat_to_index, class_to_weights = lines

text_to_class = {}
with open(test_class_list_file, 'r') as a:
    lines = map(lambda ln: ln.strip().split(' '), a.readlines())
    for ln in lines:
        file, curr_class = ln
        # text = file.split('/')[-1]
        text = re.split('[(\\\\)(\\)(\/)]', file)[-1]
        text_to_class[text] = curr_class

k = 3
max_compromise = 1
lines_to_write = []
score, total = 0, 0
with open(test_list_file, 'r') as t:
    # lines = map(lambda ln: ln.strip(), t.readlines())
    lines = map(lambda ln: ln.strip().split(' ')[0], t.readlines())
    for ln in lines:
        file = ln
        # text = file.split('/')[-1]
        text = re.split('[(\\\\)(\\)(\/)]', file)[-1]
        flat_text, word_to_count = [], {}
        with open(file, 'r') as f:
            for line in map(lambda ln: strip_and_filter_line(ln), f.readlines()):
                flat_text.extend(list(map(lambda word: p.stem(word, 0, len(word) - 1), line)))
            word_to_count = get_word_to_count(flat_text)
            fin_word_to_count = { word: count for word, count in word_to_count.items() if count >= k }
            if not fin_word_to_count:
                fin_word_to_count = get_weaker_word_to_count(word_to_count)
            sum_count = sum(fin_word_to_count.values())
            normalized_word_to_count = { word: count / sum_count for word, count in fin_word_to_count.items() }
            instance_class_to_output = { c: 0 for c in class_list }
            for c in class_list:
                for w in class_to_feat_to_index[c]:
                    if w in normalized_word_to_count:
                        index = class_to_feat_to_index[c][w]
                        instance_class_to_output[c] += class_to_weights[c][index] * normalized_word_to_count[w]
            instance_class_to_output = sorted(instance_class_to_output.items(), key = lambda x: x[1], reverse = True)
            lines_to_write.append('{} {}\n'.format(file, instance_class_to_output))
            total += 1
            if text_to_class[text] == instance_class_to_output[0][0]:
                score += 1

with open('answer', 'w') as f:
    f.writelines(lines_to_write)

In [ ]:
print(score)
print(total)